# 성인 인구조사 소득 예측

- age: 나이
- workclass: 고용 형태
- fnlwgt: 사람의 대표성을 나타내는 가중치(final weight)
- education: 교육 수준
- education.num: 교육 수준 수치
- marital.status: 결혼 상태
- occupation: 업종
- relationship: 가족 관계
- race: 인종
- sex: 성별
- capital.gain: 양도 소득
- capital.loss: 양도 손실
- hours.per.week: 주당 근무 시간
- native.country: 국적
- income: 수익 (예측해야 하는 값)

In [1]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("../input/adult-census-income/adult.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='income', null_name='?')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((26048, 15), (6513, 15), (26048, 2), (6513, 2))

# 사용자 코딩

## 라이브러리 불러오기

In [2]:
import pandas as pd

## 데이터 불러오기(생략)

In [3]:
# 시험환경에서는 아래와 같이 제공된다고 함
# import pandas as pd
# X_test = pd.read_csv("data/X_test.csv")
# X_train = pd.read_csv("data/X_train.csv")
# y_train = pd.read_csv("data/y_train.csv")

## EDA

In [4]:
# 데이터 크기 확인
X_train.shape, y_train.shape, X_test.shape

((26048, 15), (26048, 2), (6513, 15))

In [5]:
# 데이터 확인
X_train.head()

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
21851,21851,36,Private,241998,Bachelors,13,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,50,United-States
7632,7632,53,Private,103950,Masters,14,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,40,United-States
27878,27878,19,Private,203061,Some-college,10,Never-married,Tech-support,Not-in-family,White,Female,0,0,25,United-States
14121,14121,20,Private,102607,HS-grad,9,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,30,United-States
32345,32345,54,State-gov,138852,HS-grad,9,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States


In [6]:
y_train.head()

,id,income
21851,21851,>50K
7632,7632,<=50K
27878,27878,<=50K
14121,14121,<=50K
32345,32345,<=50K


In [7]:
# 타겟 수 확인
print(y_train['income'].value_counts())

<=50K    19756
>50K      6292
Name: income, dtype: int64


In [8]:
# type확인
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26048 entries, 21851 to 25716
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              26048 non-null  int64 
 1   age             26048 non-null  int64 
 2   workclass       24592 non-null  object
 3   fnlwgt          26048 non-null  int64 
 4   education       26048 non-null  object
 5   education.num   26048 non-null  int64 
 6   marital.status  26048 non-null  object
 7   occupation      24585 non-null  object
 8   relationship    26048 non-null  object
 9   race            26048 non-null  object
 10  sex             26048 non-null  object
 11  capital.gain    26048 non-null  int64 
 12  capital.loss    26048 non-null  int64 
 13  hours.per.week  26048 non-null  int64 
 14  native.country  25587 non-null  object
dtypes: int64(7), object(8)
memory usage: 3.2+ MB
None


In [9]:
# 피처 구분
# Numeric features
numeric_features = X_train.select_dtypes('int64').columns
print(numeric_features)
numeric_features = numeric_features.drop('id')
print(numeric_features)

Index(['id', 'age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss',
       'hours.per.week'],
      dtype='object')
Index(['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss',
       'hours.per.week'],
      dtype='object')


In [10]:
# Categorical features
cat_features = X_train.select_dtypes('object').columns
print(cat_features)

Index(['workclass', 'education', 'marital.status', 'occupation',
       'relationship', 'race', 'sex', 'native.country'],
      dtype='object')


In [11]:
X_train[numeric_features].describe()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,26048.000000,2.604800e+04,26048.000000,26048.000000,26048.000000,26048.000000
mean,38.610335,1.895741e+05,10.082118,1081.193796,88.477695,40.420224
std,13.628346,1.043848e+05,2.574608,7404.962675,404.689981,12.354707
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.182472e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.785755e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.365968e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [12]:
print(X_train[cat_features].describe())

       workclass education      marital.status       occupation relationship  \
count      24592     26048               26048            24585        26048   
unique         8        16                   7               14            6   
top      Private   HS-grad  Married-civ-spouse  Exec-managerial      Husband   
freq       18160      8408               11987             3323        10558   

         race    sex native.country  
count   26048  26048          25587  
unique      5      2             41  
top     White   Male  United-States  
freq    22270  17400          23381  


## 결측치 처리

In [13]:
print(X_train.isnull().sum())

id                   0
age                  0
workclass         1456
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1463
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     461
dtype: int64


In [14]:
print(X_test.isnull().sum())

id                  0
age                 0
workclass         380
fnlwgt              0
education           0
education.num       0
marital.status      0
occupation        380
relationship        0
race                0
sex                 0
capital.gain        0
capital.loss        0
hours.per.week      0
native.country    122
dtype: int64


In [15]:
print(X_train['workclass'].value_counts())

Private             18160
Self-emp-not-inc     2049
Local-gov            1648
State-gov            1037
Self-emp-inc          909
Federal-gov           770
Without-pay            12
Never-worked            7
Name: workclass, dtype: int64


In [16]:
print(X_train['occupation'].value_counts())

Exec-managerial      3323
Prof-specialty       3306
Craft-repair         3296
Adm-clerical         3037
Sales                2898
Other-service        2624
Machine-op-inspct    1584
Transport-moving     1257
Handlers-cleaners    1080
Farming-fishing       786
Tech-support          746
Protective-serv       521
Priv-house-serv       119
Armed-Forces            8
Name: occupation, dtype: int64


In [17]:
print(X_train['native.country'].value_counts())

United-States                 23381
Mexico                          516
Philippines                     158
Germany                         108
Canada                           88
Puerto-Rico                      87
El-Salvador                      76
India                            73
Cuba                             73
England                          69
Italy                            63
South                            62
Jamaica                          59
Vietnam                          57
China                            57
Guatemala                        54
Dominican-Republic               51
Japan                            49
Poland                           47
Columbia                         44
Taiwan                           37
Haiti                            37
Iran                             34
Portugal                         32
Peru                             29
Nicaragua                        27
Ecuador                          25
Greece                      

- 결측치는 최빈값과 차이가 크면 최빈값**(mode())** 으로 값이 비슷하면 별도의 값으로 대체함

In [18]:
def data_fillna(df):
    df['workclass'] = df['workclass'].fillna(df['workclass'].mode()[0])
    df['occupation'] = df['occupation'].fillna('null')
    df['native.country'] = df['native.country'].fillna(df['native.country'].mode()[0])
    return df

X_train = data_fillna(X_train)
X_test = data_fillna(X_test)

print(X_test.isnull().sum())

id                0
age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
dtype: int64


In [19]:
print(X_train.isnull().sum())

id                0
age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
dtype: int64


## 피처엔지니어링

In [21]:
# 라벨인코딩
from sklearn.preprocessing import LabelEncoder

all_df = pd.concat([X_train.assign(ind='train'), X_test.assign(ind='test')])
le = LabelEncoder()

all_df[cat_features] = all_df[cat_features].apply(le.fit_transform)
all_df.head()

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,ind
21851,21851,36,3,241998,9,13,2,2,0,4,1,0,0,50,38,train
7632,7632,53,3,103950,12,14,0,9,1,4,0,0,0,40,38,train
27878,27878,19,3,203061,15,10,4,12,1,4,0,0,0,25,38,train
14121,14121,20,3,102607,11,9,4,5,3,4,1,0,0,30,38,train
32345,32345,54,6,138852,11,9,2,9,0,4,1,0,0,40,38,train


In [24]:
X_train = all_df[all_df['ind'] == 'train']
X_train = X_train.drop('ind', axis=1)
print(X_train.head())

          id  age  workclass  fnlwgt  education  education.num  \
21851  21851   36          3  241998          9             13   
7632    7632   53          3  103950         12             14   
27878  27878   19          3  203061         15             10   
14121  14121   20          3  102607         11              9   
32345  32345   54          6  138852         11              9   

       marital.status  occupation  relationship  race  sex  capital.gain  \
21851               2           2             0     4    1             0   
7632                0           9             1     4    0             0   
27878               4          12             1     4    0             0   
14121               4           5             3     4    1             0   
32345               2           9             0     4    1             0   

       capital.loss  hours.per.week  native.country  
21851             0              50              38  
7632              0              40   

In [25]:
X_test = all_df[all_df['ind'] == 'test']
X_test = X_test.drop('ind', axis=1)
print(X_test.head())

          id  age  workclass  fnlwgt  education  education.num  \
20901  20901   58          3  114495         11              9   
14170  14170   46          3  247043         11              9   
1776    1776   67          1  103315         12             14   
30428  30428   18          3  165532         15             10   
8602    8602   26          6   58039         15             10   

       marital.status  occupation  relationship  race  sex  capital.gain  \
20901               2           3             0     4    1             0   
14170               2          13             0     4    1             0   
1776                4           3             2     4    0         15831   
30428               4          11             3     4    1             0   
8602                2           7             0     4    1             0   

       capital.loss  hours.per.week  native.country  
20901             0              40              38  
14170             0              40   

In [27]:
# 스케일링
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test[numeric_features] = scaler.transform(X_test[numeric_features])

In [35]:
# target값 변경
y = (y_train['income'] != '<=50K').astype(int)
y

21851    1
7632     0
27878    0
14121    0
32345    0
        ..
2669     1
17536    0
6201     0
27989    0
25716    0
Name: income, Length: 26048, dtype: int64

## 검증용 데이터 분리

In [36]:
# 학습용 데이터와 검증용 데이터로 구분
from sklearn.model_selection import train_test_split

X_tr,X_val,y_tr,y_val = train_test_split(X_train, y, test_size = 0.2, random_state = 2023)
print(X_tr.shape,X_val.shape,y_tr.shape,y_val.shape)

(20838, 15) (5210, 15) (20838,) (5210,)


In [38]:
print(X_tr.head())

          id       age  workclass    fnlwgt  education  education.num  \
6618    6618  0.328767          3  0.164871          9       0.800000   
25663  25663  0.410959          3  0.146035         15       0.600000   
30950  30950  0.315068          3  0.102591         11       0.533333   
23391  23391  0.150685          3  0.105304         11       0.533333   
30697  30697  0.315068          3  0.021343         11       0.533333   

       marital.status  occupation  relationship  race  sex  capital.gain  \
6618                0           3             1     4    0           0.0   
25663               0           0             4     2    0           0.0   
30950               4           0             1     4    0           0.0   
23391               2           3             5     4    0           0.0   
30697               4           6             1     4    1           0.0   

       capital.loss  hours.per.week  native.country  
6618            0.0        0.551020              3

In [39]:
# id 삭제
X_tr = X_tr.drop('id', axis=1)
X_val = X_val.drop('id', axis=1)
X_tr.shape, X_val.shape

((20838, 14), (5210, 14))

In [40]:
# id 삭제된 것 확인
X_tr.head(1)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
6618,0.328767,3,0.164871,9,0.8,0,3,1,4,0,0.0,0.0,0.55102,38


## 모델 & 평가

In [42]:
# 의사결정나무
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dtc = DecisionTreeClassifier()
dtc.fit(X_tr, y_tr)
pred = dtc.predict(X_val)

print('accuracy score: ', (accuracy_score(y_val, pred)))

accuracy score:  0.8069097888675624


In [44]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_tr, y_tr)
pred = rfc.predict(X_val)

print('accuracy score: ', (accuracy_score(y_val, pred)))

accuracy score:  0.8558541266794626


In [45]:
# test데이터 예측 (pop을 활용하면 값을 넘겨주고 삭제 됨)
X_test_id = X_test.pop('id')
pred = rfc.predict(X_test)

In [49]:
# csv생성
output = pd.DataFrame({'id':X_test_id, 'income':pred})
output.to_csv('00600.csv', index =False)
print(output.head())

          id  income
20901  20901       1
14170  14170       0
1776    1776       1
30428  30428       0
8602    8602       1


## 채점 (수험자는 확인 불가)

In [47]:
y_test = (y_test['income'] != '<=50K').astype(int)
from sklearn.metrics import accuracy_score
print('accuracy score:', (accuracy_score(y_test, pred)))

accuracy score: 0.702287732227852
